In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.datasets import mnist

In [3]:
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [4]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()

In [5]:
X_train = X_train.astype('float32') / 255.0
X_test = X_test.astype('float32') / 255.0

In [6]:
X_train.shape

(60000, 28, 28)

In [7]:
class CNNBlock(layers.Layer):
    def __init__(self, output_channels, kernel_size=3):
        super(CNNBlock, self).__init__()
        self.conv = layers.Conv2D(output_channels, kernel_size, padding='same')
        self.bn = layers.BatchNormalization()

    def call(self, input, training=False):
        x = self.conv(input)
        x = self.bn(x, training=training)
        x = tf.keras.activations.relu(x)
        return x

In [8]:
model = keras.Sequential(
    [
        layers.Input((28,28,1)),
        CNNBlock(32),
        layers.MaxPooling2D(),
        CNNBlock(64),
        layers.MaxPooling2D(),
        CNNBlock(128),
        layers.Flatten(),
        layers.Dense(10)
    ]
)

In [9]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 cnn_block (CNNBlock)        (None, 28, 28, 32)        448       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 14, 14, 32)       0         
 )                                                               
                                                                 
 cnn_block_1 (CNNBlock)      (None, 14, 14, 64)        18752     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 7, 7, 64)         0         
 2D)                                                             
                                                                 
 cnn_block_2 (CNNBlock)      (None, 7, 7, 128)         74368     
                                                                 
 flatten (Flatten)           (None, 6272)              0

In [10]:
model.compile(
    loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer = keras.optimizers.Adam(learning_rate=0.001),
    metrics = ['accuracy']
)

In [11]:
model.fit(
    X_train, y_train,
    batch_size = 32,
    epochs = 5,
    validation_data = (X_test, y_test)
)

Epoch 1/5
1875/1875 [==============================] - 42s 16ms/step - loss: 0.1111 - accuracy: 0.9666 - val_loss: 0.0660 - val_accuracy: 0.9812
Epoch 2/5
1875/1875 [==============================] - 74s 39ms/step - loss: 0.0488 - accuracy: 0.9856 - val_loss: 0.0519 - val_accuracy: 0.9848
Epoch 3/5
1875/1875 [==============================] - 20s 11ms/step - loss: 0.0307 - accuracy: 0.9907 - val_loss: 0.0262 - val_accuracy: 0.9910
Epoch 4/5
1875/1875 [==============================] - 19s 10ms/step - loss: 0.0258 - accuracy: 0.9916 - val_loss: 0.0286 - val_accuracy: 0.9915
Epoch 5/5
1875/1875 [==============================] - 18s 10ms/step - loss: 0.0179 - accuracy: 0.9939 - val_loss: 0.0308 - val_accuracy: 0.9897


### Building Resnet

In [12]:
class CNNBlock(layers.Layer):
    def __init__(self, output_channels, kernel_size=3):
        super(CNNBlock, self).__init__()
        self.conv = layers.Conv2D(output_channels, kernel_size, padding='same')
        self.bn = layers.BatchNormalization()

    def call(self, input, training=False):
        x = self.conv(input)
        x = self.bn(x, training=training)
        x = tf.keras.activations.relu(x)
        return x

In [13]:
class ResBlock(layers.Layer):
    def __init__(self, channels):
        super(ResBlock, self).__init__()
        self.channels = channels
        self.cnn1 = CNNBlock(channels[0])
        self.cnn2 = CNNBlock(channels[1])
        self.cnn3 = CNNBlock(channels[2])
        self.pooling = layers.MaxPooling2D()
        self.identity_mapping = layers.Conv2D(channels[1], 1, padding='same')

    def call(self, input_tensor, training=False):
        x = self.cnn1(input_tensor, training=training)
        x = self.cnn2(x, training=training)
        x = self.cnn3(x + self.identity_mapping(input_tensor), training=training)
        x = self.pooling(x)
        return x

In [14]:
class ResNet_Like(keras.Model):
    def __init__(self, num_classes=10):
        super(ResNet_Like, self).__init__()
        self.block1 = ResBlock([32, 32, 64])
        self.block2 = ResBlock([128, 128, 256])
        self.block3 = ResBlock([128, 256, 512])
        self.pool = layers.GlobalAveragePooling2D()
        self.classifier = layers.Dense(num_classes)

    def call(self, input_tensor, training=False):
        x = self.block1(input_tensor, training=training)
        x = self.block2(x, training=training)
        x = self.block3(x, training=training)
        x = self.pool(x, training=training)
        x = self.classifier(x)
        return x

    def model(self):
        x = keras.Input(shape=(28,28,1))
        return keras.Model(inputs=[x], outputs=self.call(x))

In [15]:
model = ResNet_Like().model()

In [16]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 28, 28, 1)]       0         
                                                                 
 res_block (ResBlock)        (None, 14, 14, 64)        28640     
                                                                 
 res_block_1 (ResBlock)      (None, 7, 7, 256)         526976    
                                                                 
 res_block_2 (ResBlock)      (None, 3, 3, 512)         1839744   
                                                                 
 global_average_pooling2d (G  (None, 512)              0         
 lobalAveragePooling2D)                                          
                                                                 
 dense_1 (Dense)             (None, 10)                5130      
                                                             

In [17]:
model.compile(
    optimizer=keras.optimizers.Adam(),
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"],
)

In [18]:
model.fit(
    X_train, y_train,
    batch_size=32,
    epochs=10,
    validation_data=(X_test, y_test)
)

Epoch 1/10
1875/1875 [==============================] - 70s 36ms/step - loss: 0.1536 - accuracy: 0.9505 - val_loss: 0.0489 - val_accuracy: 0.9850
Epoch 2/10
1875/1875 [==============================] - 60s 32ms/step - loss: 0.0439 - accuracy: 0.9869 - val_loss: 0.0238 - val_accuracy: 0.9924
Epoch 3/10
1875/1875 [==============================] - 54s 29ms/step - loss: 0.0359 - accuracy: 0.9896 - val_loss: 0.0246 - val_accuracy: 0.9921
Epoch 4/10
1875/1875 [==============================] - 62s 33ms/step - loss: 0.0283 - accuracy: 0.9913 - val_loss: 0.0231 - val_accuracy: 0.9925
Epoch 5/10
1875/1875 [==============================] - 70s 37ms/step - loss: 0.0227 - accuracy: 0.9931 - val_loss: 0.0301 - val_accuracy: 0.9919
Epoch 6/10
1875/1875 [==============================] - 80s 43ms/step - loss: 0.0202 - accuracy: 0.9938 - val_loss: 0.0377 - val_accuracy: 0.9900
Epoch 7/10
1875/1875 [==============================] - 76s 41ms/step - loss: 0.0174 - accuracy: 0.9950 - val_loss: 0.0306 -

In [19]:
model.save("models/saved_model/")

INFO:tensorflow:Assets written to: models/saved_model/assets


INFO:tensorflow:Assets written to: models/saved_model/assets
